In [21]:
import numpy as np
import pandas as pd
import torch
from math import factorial

In [16]:
# Read data from file and seperate x and y
def loader(filename):
    raw_df = pd.read_csv(filename,header=None,sep = '\s+|:',engine='python')
    y = torch.tensor(raw_df[0].values)
    x = torch.tensor(raw_df[raw_df.columns[2::2]].values)
    return x,y

In [17]:
x,y = loader('diabetes')

In [20]:
L = 1.0 # range
N = 10 # degree

In [18]:
def obj_pos(i,t,prod):
    fpt = 0.0 # p stands for positive
    gfpt = 0.0 # g stands for gradient
    fpt = (L/2+prod)**i 
    gfpt = i*(L/2+prod)**(i-1) # no xt yet!
    return fpt,gfpt               

In [22]:
# Can be rewrite as a lambda function
def hinge(x):
    phi = max(0,x)
    return phi

In [23]:
def comb(n, k):
    return factorial(n) / factorial(k) / factorial(n - k)

In [25]:
def coef(loss,i,k):
    # compute forward difference
    delta = 0.0
    for j in range(k+1):
        delta += (-1)**(k-j)*comb(k,j)*loss(i/N)
    # compute coefficient
    beta = comb(N,k)*comb(k,i)*(N+1)*delta/(2*L)**k
    return beta

In [27]:
def obj_neg(loss,i,t,prod):
    fnt = 0.0 # n stands for negative
    gfnt = 0.0
    for k in range(i,N+1):
        beta = coef(loss,i,k)
        fnt += beta*(L/2-prod)**(k-i)
        gfnt += beta*(k-i)*(L/2-prod)**(k-i-1)  # no xt yet!
    return fnt,gfnt

In [6]:
def p_hat(t,yt,ptm1):
    # Approximate probability
    pt = ((t-1)*ptm1 + (yt+1)/2)/t # m stands for minus
    return pt

In [7]:
def a_hat(t,fpt,yt,ptm1,atm1):
    # Approximate a
    at = (fpt*((yt+1)/2) + (t-1)*ptm1*atm1)/t # do not update pt yet!
    return at

In [8]:
def b_hat(t,fmt,yt,ptm1,btm1):
    # Approximate b
    bt = (fmt*((-yt+1)/2) + (t-1)*(1-ptm1)*btm1)/t # indicator of y=-1!
    return bt

In [9]:
def alpha_step(t,at,bt):
    alphat = at + bt
    return alphat

In [10]:
def w_grad(i,k,t,wt,fpt,fnt,yt,pt,at,bt,alphat):
    gfpt = i*(L/2+wt*xt)**(i-1)*xt # g stands for gradient
    gfnt = coef(loss,i,k)
    return wt

In [11]:
def a_grad(t,fpt,fmt,yt,pt,at,bt,alphat):
    # Define the gradient at current t
    at = 0.0 # to be determined
    return at

In [12]:
def b_grad(t,fpt,fmt,yt,pt,at,bt,alphat):
    bt = 0.0
    return bt

In [26]:
def SPAM(T,wt=0.0,pt=0.0,at=0.0,bt=0.0):
    '''
    This is it!
    '''
    # Initialize W
    W = torch.empty(T)
    
    # Loop
    for t in range(1,T+1):
        # Compute scoring function
        fpt = obj_pos(t,wt,x[t])
        fmt = obj_neg(t,wt,x[t])
        # Update at,bt without update pt yet
        at = a_hat(t,fpt,y[t],pt,at)
        bt = b_hat(t,fmt,y[t],pt,bt)
        # Update pt
        pt = p_hat(t,y[t],pt)
        # Compute real at,bt,alphat
        at *= 1/pt
        bt *= 1/(1-pt)
        alphat = at + bt
        # Compute gradient
        gt = F_grad(t,fpt,fmt,y[t],pt,at,bt,alphat)
        # Gradient descent
        wt += -1/t * gt
        
        # Record wt
        W[t-1] = wt
    return W